In [13]:
import os
import pandas as pd
from datetime import datetime, time

In [14]:
#Run this cell to get ONE parquet file of ALL cleaned data stored in the folder all_cleaned_data.

def get_cleaned_df():
    folder_path = "cleaned_data/"
    
    os.makedirs("all_cleaned_data", exist_ok=True)
    
    # Get all parquet files in the folder
    parquet_files = [f for f in os.listdir(folder_path) if f.endswith('.parquet')]
    
    # Load and concatenate them into a single DataFrame
    dataframes = [pd.read_parquet(os.path.join(folder_path, file)) for file in parquet_files]
    
    # Combine all dataframes
    df = pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()
    
    df.to_parquet("all_cleaned_data/all_cleaned_data.parquet", index=False)

get_cleaned_df()

In [15]:
#return all cleaned data file as a dataframe
df = pd.read_parquet("all_cleaned_data/all_cleaned_data.parquet")
df.head()

,VendorID,pickup_datetime,dropoff_datetime,trip_duration,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,total_amount,tip_amount,tolls_amount,congestion_surcharge
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,19.800000,1.0,1.72,1.0,186,79,2,17.7,22.70,0.00,0.0,2.5
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,6.600000,1.0,1.80,1.0,140,236,1,10.0,18.75,3.75,0.0,2.5
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,17.916667,1.0,4.70,1.0,236,79,1,23.3,31.30,3.00,0.0,2.5
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,8.300000,1.0,1.40,1.0,79,211,1,10.0,17.00,2.00,0.0,2.5
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,6.100000,1.0,0.80,1.0,211,148,1,7.9,16.10,3.20,0.0,2.5


In [16]:
def DOPU_given_timerange(df, zone, start, end, isDropoff=True):
    """
    input: 
        df: dataframe to extract number of dropoffs from
        zone: integer zone id
        start: time of start (pandas time object)
        end: time of end (pandas time object)
        isDropoff: bool. If true, 
        
    returns:
        integer- average number of dropoffs/pickups made in the zone during the time range
    """
    
    
    #given a zone id, a start time, and end time, find the average number of dropoffs/pickups made during this time
    df_zone = df[df["DOLocationID"] == zone].copy() if isDropoff else df[df["PULocationID"] == zone].copy()
    
    #remove date from datetime object
    df_zone["time"] = df_zone["dropoff_datetime"].dt.time if isDropoff else df_zone["pickup_datetime"].dt.time
    
    #filter by times within the given time range
    df_filtered = df_zone[(df_zone["time"] >= start) & (df_zone["time"] <= end)]
    
    #sum all dropoff/pickups
    dropoffs_per_day = df_filtered.groupby(df_filtered["dropoff_datetime"].dt.date).size()
    
    #return mean dropoffs/pickups
    return int(dropoffs_per_day.mean()) if not dropoffs_per_day.empty else 0



In [17]:
def DOPU_given_one_timerange(df, zone, time, timerange, isDropoff=True):
    """
    input: 
        df: dataframe to extract number of dropoffs from
        zone: integer zone id
        time: datetime.time object- the time to check
        timerange: pd.Deltatime object- the timerange to check within
        isDropoff: bool. If true, returns num of dropoffs. If false, returns num of pickups
        
    returns:
        integer- average number of dropoffs/pickups made in the zone during the time range
    """
    
    #given a zone id, a start time, and end time, find the average number of dropoffs/pickups made during this time
    df_zone = df[df["DOLocationID"] == zone].copy() if isDropoff else df[df["PULocationID"] == zone].copy()
    
    ref_date = datetime.combine(datetime.today(), time)
    #remove date from datetime object
    df_zone["time"] = df_zone["dropoff_datetime"].dt.time if isDropoff else df_zone["pickup_datetime"].dt.time
    
    start = (ref_date - timerange).time()
        
    end = (ref_date + timerange).time()
    
    #filter by times within the given time range

    df_filtered = df_zone[(df_zone["time"] >= start) & (df_zone["time"] <= end)] if (start < end) else df_zone[(df_zone["time"] >= start) | (df_zone["time"] <= end)]
    
    #sum all dropoff/pickups
    dropoffs_per_day = df_filtered.groupby(df_filtered["dropoff_datetime"].dt.date).size()
    
    #return mean dropoffs/pickups
    return int(dropoffs_per_day.mean()) if not dropoffs_per_day.empty else 0



In [30]:
import numpy as np

tim = time(9, 0, 0)
timerange = pd.Timedelta(hours=4.0)
n = DOPU_given_one_timerange(df, 10, tim, timerange, False)
print(n)


start = time(5, 0, 0)
end = time(13, 0, 0)
m = DOPU_given_timerange(df, 10, start, end, False)
print(m)
max = 0
for LocationID in np.sort(df["DOLocationID"].unique()):
    print("ID:", LocationID)
    count = DOPU_given_one_timerange(df, LocationID, tim, timerange, False)
    print("Count:", count)
    if max < count:
        max = count
print(max)
    

6
6
ID: 1
Count: 1
ID: 2
Count: 1
ID: 3
Count: 1
ID: 4
Count: 8
ID: 5
Count: 0
ID: 6
Count: 1
ID: 7
Count: 12
ID: 8
Count: 1
ID: 9
Count: 1
ID: 10
Count: 6
ID: 11
Count: 1
ID: 12
Count: 8
ID: 13
Count: 153
ID: 14
Count: 1
ID: 15
Count: 1
ID: 16
Count: 1
ID: 17
Count: 1
ID: 18
Count: 1
ID: 19
Count: 1
ID: 20
Count: 1
ID: 21
Count: 1
ID: 22
Count: 1
ID: 23
Count: 1
ID: 24
Count: 92
ID: 25
Count: 5
ID: 26
Count: 1
ID: 27
Count: 1
ID: 28
Count: 2
ID: 29
Count: 1
ID: 30
Count: 1
ID: 31
Count: 1
ID: 32
Count: 1
ID: 33
Count: 9
ID: 34
Count: 1
ID: 35
Count: 1
ID: 36
Count: 1
ID: 37
Count: 1
ID: 38
Count: 1
ID: 39
Count: 1
ID: 40
Count: 4
ID: 41
Count: 71
ID: 42
Count: 20
ID: 43
Count: 448
ID: 44
Count: 1
ID: 45
Count: 22
ID: 46
Count: 1
ID: 47
Count: 1
ID: 48
Count: 633
ID: 49
Count: 1
ID: 50
Count: 180
ID: 51
Count: 1
ID: 52
Count: 7
ID: 53
Count: 1
ID: 54
Count: 1
ID: 55
Count: 1
ID: 56
Count: 1
ID: 57
Count: 1
ID: 58
Count: 1
ID: 59
Count: 1
ID: 60
Count: 1
ID: 61
Count: 1
ID: 62
Count: 1
